In [ ]:
import json
from fastai.vision.all importorch.cuda.is_available()t *

default_device()

In [ ]:
torch.cuda.is_available(), torch.version.cuda

In [ ]:
data_base = Path("../../School/Senior Design").resolve()
data_dir = data_base / "train_mini"
data_json = data_base / "train_mini.json"
data_dir, data_json, data_dir.exists(), data_json.exists()

In [ ]:
def get_annotes(fname):
    annot_dict = json.load(open(fname))
    id2images, id2cats = {}, collections.defaultdict(list)
    classes = {o['id']: o['name'] for o in annot_dict['categories']}
    for o in annot_dict['annotations']:
        id2cats[o['image_id']].append(classes[o['category_id']])
    id2images = {o['id']: data_base / o['file_name'] for o in annot_dict['images']}
    ids = list(id2images.keys())
    return [id2images[k] for k in ids], [id2cats[k] for k in ids]

imgs, labels = get_annotes(data_json)
len(imgs), len(labels)

In [ ]:
#imgs = [[str(fn)] for fn in imgs]
labels = [label[0] for label in labels]
len(imgs), len(imgs), imgs[0], labels[0]

In [ ]:
dls = ImageDataLoaders.from_lists(data_base, imgs, labels, valid_pct=0.2,
                                   bs=64, item_tfms=Resize(460),
                                   batch_tfms=aug_transforms(size=224))

In [ ]:
dls.show_batch()

In [ ]:
learn = vision_learner(dls, resnet18, metrics=error_rate)

In [ ]:
learn.lr_find()

In [ ]:
learn.fine_tune(5, 3e-3)

In [ ]:
learn.show_results()